In [1]:
league = "Ligue_1"
season = "2024"

In [2]:
import pandas as pd
import requests
import re
from unidecode import unidecode
from sqlalchemy.exc import SQLAlchemyError

In [3]:
import understatapi
client = understatapi.UnderstatClient()

In [4]:
from sqlalchemy import create_engine
import psycopg2

# Database connection settings (replace with your credentials)
db_config = {
    'dbname': 'understat_shots_db',
    'user': 'ichadha',
    'password': 'ichadhapg',
    'host': 'localhost',
    'port': 5432,
}

# Create a connection engine
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")

#with engine.connect() as conn:
#    conn.execute('TRUNCATE TABLE understat_shots_tb RESTART IDENTITY CASCADE;')

def save_to_postgres(dataframe, table_name):
    try:
        # Append data to the PostgreSQL table
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Data successfully saved to {table_name} table.")
    except Exception as e:
        print(f"Error saving data to PostgreSQL: {e}")


def upsert_to_postgres(df, table_name, primary_key, engine):
    try:
        with engine.connect() as conn:
            # Begin a transaction
            with conn.begin():
                # Create a temporary table for the upsert operation
                temp_table_name = f"{table_name}_temp"
                df.to_sql(temp_table_name, con=conn, if_exists='replace', index=False)
                
                # Construct the merge query using a WITH clause
                quoted_columns = ["X", "Y","xG","shotType","lastAction"]
                update_columns = ", ".join([f'"{col}" = EXCLUDED."{col}"' if col in quoted_columns else f'{col} = EXCLUDED.{col}' for col in df.columns if col != primary_key])
                insert_columns = ", ".join([f'"{col}"' if col in quoted_columns else col for col in df.columns])
                insert_values = ", ".join([f':{col}' for col in df.columns])

                upsert_query = f"""
                INSERT INTO {table_name} ({insert_columns})
                SELECT {insert_columns} FROM {temp_table_name}
                ON CONFLICT ({primary_key})
                DO UPDATE SET {update_columns};
                """
                
                # Execute the merge query
                conn.execute(upsert_query)
                
                # Drop the temporary table
                conn.execute(f"DROP TABLE IF EXISTS {temp_table_name};")
                
            print(f"Upsert to table '{table_name}' completed successfully.")
    
    except SQLAlchemyError as e:
        print(f"Error during upsert: {str(e)}")


In [5]:
## EPL , Bundesliga , Serie_A , Ligue_1 , La_Liga
#league = "EPL"
#season = "2024"

league_data = client.league(league=league).get_match_data(season=season)

In [6]:
matches = []

for match in league_data:
    match_data = {
        'id': match['id'],
        'home_team': match['h']['title'],
        'away_team': match['a']['title'],
        'home_goals': match['goals']['h'],
        'away_goals': match['goals']['a'],
        'home_xG': match['xG']['h'],
        'away_xG': match['xG']['a'],
        'datetime': match['datetime']
        #'forecast': match['forecast']
    }
    matches.append(match_data)

# Convert the list of dictionaries into a DataFrame
matches_df = pd.DataFrame(matches)

In [7]:
all_shot_data = []

import datetime

current_timestamp = datetime.datetime.now().timestamp()

for index,row in matches_df.iterrows():
    match_datetime = pd.to_datetime(row['datetime']).timestamp()
    
    if match_datetime <= current_timestamp:
        #print(row['id'])
        try:
            shot_data = client.match(match=row['id']).get_shot_data()
        except Exception as e:
            print(e)
        all_shot_data.append(shot_data)

In [8]:
# Initialize an empty list to hold individual shot records
compiled_shot_data = []

# Loop through each match in `all_shot_data`
for match in all_shot_data:
    # Get home shots and away shots from the match
    home_shots = match.get('h', [])
    away_shots = match.get('a', [])
    
    # Add a column to indicate if it's a home or away shot, then extend our list
    for shot in home_shots:
        shot['h_a'] = 'h'  # Indicate as home shot
        compiled_shot_data.append(shot)
    
    for shot in away_shots:
        shot['h_a'] = 'a'  # Indicate as away shot
        compiled_shot_data.append(shot)

# Convert the list of shot records to a DataFrame
compiled_shot_df = pd.DataFrame(compiled_shot_data)

In [9]:
compiled_shot_df['league'] = league

In [10]:
compiled_shot_df = compiled_shot_df.drop_duplicates()

In [11]:
compiled_shot_df['id'] = compiled_shot_df['id'].astype(int)
compiled_shot_df['minute'] = compiled_shot_df['minute'].astype(int)
compiled_shot_df['X'] = pd.to_numeric(compiled_shot_df['X'], errors='coerce')
compiled_shot_df['Y'] = pd.to_numeric(compiled_shot_df['Y'], errors='coerce')
compiled_shot_df['xG'] = pd.to_numeric(compiled_shot_df['xG'], errors='coerce')
compiled_shot_df['player_id'] = compiled_shot_df['player_id'].astype(int)
compiled_shot_df['season'] = compiled_shot_df['season'].astype(int)
compiled_shot_df['match_id'] = compiled_shot_df['match_id'].astype(int)
compiled_shot_df['h_goals'] = compiled_shot_df['h_goals'].astype(int)
compiled_shot_df['a_goals'] = compiled_shot_df['a_goals'].astype(int)
compiled_shot_df['date'] = pd.to_datetime(compiled_shot_df['date'], errors='coerce')

In [12]:
#compiled_shot_df.to_csv(f'C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/Data/{league}/2024-25/shot_data.csv')
#save_to_postgres(compiled_shot_df, 'understat_shots_tb')
upsert_to_postgres(compiled_shot_df, 'understat_shots_tb','id',engine)

Error during upsert: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "ichadha"

(Background on this error at: https://sqlalche.me/e/14/e3q8)
